In [1]:
!pip install selenium==4.12.0

Defaulting to user installation because normal site-packages is not writeable
  Using cached selenium-4.12.0-py3-none-any.whl (9.4 MB)
     -------------------------------------- 460.2/460.2 kB 3.2 MB/s eta 0:00:00
  Using cached sniffio-1.3.0-py3-none-any.whl (10 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)


In [2]:
!pip install PyPDF2 boto3

Defaulting to user installation because normal site-packages is not writeable
  Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)


In [3]:
from time import sleep,time
from selenium import webdriver
from selenium.webdriver import *
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import getpass

import os
from glob import glob
from PyPDF2 import PdfMerger
import json
import boto3

In [31]:
def ssm_get_qslogin_info():
    secret_name = "quicksight/pdf/user"
    region_name = "ap-northeast-2"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        # For a list of exceptions thrown, see
        # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        raise e

    # Decrypts secret using the associated KMS key.
    secret = json.loads(get_secret_value_response['SecretString'])
    return secret['quicksight_user'],secret['quicksight_pw']
    # Your code goes here.

In [27]:
## Clean dir
def clean_dir(result_dir,glob_str="*.pdf"):
    filelist = glob(os.path.join(result_dir, glob_str))
    for f in filelist:
        os.chmod(f, 0o777)
        os.remove(f)

## function download
# Volkswagenquicksightpdf132#
def quicksigt_login(driver,qid,qpwd):
    driver.get("https://us-east-1.quicksight.aws.amazon.com/sn/dashboards/a7adfe68-ab7e-45f4-a988-94bc41cf9c53")
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID,'account-name-input')))

    driver.find_element(By.ID,'account-name-input').send_keys('volkswagenkorea')
    driver.find_element(By.ID,'account-name-submit-button').click()
    
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "username-input")))
    driver.find_element(By.ID,'username-input').send_keys(qid)
    driver.find_element(By.ID,'username-submit-button').click()

    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "awsui-input-0")))

    driver.find_element(By.ID,'awsui-input-0').send_keys(qpwd)
    driver.find_element(By.ID,'password-submit-button').click()

    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//button[@aria-label="Export"]')))
    return driver

def quicksight_download(driver,total_page,url):
    driver.get(url)
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//button[@aria-label="Export"]')))
    driver.find_element(By.XPATH, '//button[@aria-label="Export"]').click()
    driver.find_element(By.XPATH, '//li[@data-automation-id="navbar_export_pane_toggle_open"]').click()
    sleep(1)
    for i in range(1,total_page+1):
        driver.find_element(By.XPATH,f'//*[@id="application-content"]/div/div/div[2]/div[1]/div[2]/div/div[1]/div/div/div/div[{i}]').click()
        sleep(1)
        driver.find_element(By.XPATH, '//button[@aria-label="내보내기"]').click()
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//li[@title="PDF 생성"]')))
        driver.find_element(By.XPATH, '//li[@title="PDF 생성"]').click()
        sleep(1)
    
    while True:
        ready_elem = len(driver.find_elements(By.CSS_SELECTOR,"h6"))
        try:
            driver.find_element(By.ID,'alertCloseButton').click()
            sleep(0.3)
        except:
               pass
        finally:
            print(" "*50,end='\r')
            print(f"Wait for Donwload! ({ready_elem}/{total_page}) : {round((ready_elem/total_page)*100,2)}%",end='\r')
            if ready_elem == total_page: 
                print("\nDownload Ready!") 
                break
    
    for i in range(1,total_page+1):
        driver.find_element(By.XPATH, f'//*[@id="vega-right-pane"]/div/div/div/div[{i}]/div[2]/p[2]/div').click()
        sleep(0.2)
    
    return driver

def download_wait(md_file_count):
        while True:
            clean_dir(result_dir,"* ([1-9]).pdf")
            download_file_count = len(glob(os.path.join(result_dir, "*p*.pdf")))
            if download_file_count == md_file_count:
                print("Download Complete")
                break
        return

def merge_pdf(tot_page,pdf_file_name,result_dir):
    merger = PdfMerger()
    for i in range(1,tot_page+1):
        merger.append(glob(f'{result_dir}/{i}p*'+'.pdf')[0])

    merger.write(f"{result_dir}/merged/"+pdf_file_name + ".pdf")
    merger.close()

result_dir=r"C:\Data_Files"

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_experimental_option("prefs", {
  "download.default_directory": result_dir
  })

clean_dir(result_dir)

#dashboard_url = input("url : ")
# quicksight_id = input("ID : ")
# pswd = getpass.getpass('Password:')
quicksight_id = "pdf_test"
pswd = "Pdf_test1"

#login
service = webdriver.ChromeService("chromedriver.exe")
driver = webdriver.Chrome(service=service,options=options)
driver.implicitly_wait(3)
driver = quicksigt_login(driver,quicksight_id,pswd)

start = time()

##Dashboard 1_1 Page Download   

d1_url="https://us-east-1.quicksight.aws.amazon.com/sn/dashboards/8e2c6dc2-fd18-4ad2-ab10-942149c252ab"
page_d1=18

from selenium.common.exceptions import *
errors = [NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException]
wait = WebDriverWait(driver, timeout=5, poll_frequency=.2, ignored_exceptions=errors)
wait.until(lambda d : d.find_element(By.XPATH,f"/html/body/div[4]/div[2]/div/div/div/div[2]/div[1]/div[2]/div/div[1]/div/div/div/div[3]"))

#driver = quicksight_download(driver,page_d1,d1_url)
# download_wait(page_d1)

# ##Dashboard 1_2 Page Download   

# d2_url="https://us-east-1.quicksight.aws.amazon.com/sn/dashboards/7b03e7cf-3485-4610-ab1f-b4a3a7c6ca1e"
# page_d2= 7
# driver = quicksight_download(driver,page_d2,d2_url)
# download_wait(page_d1+page_d2)

# ##Cover Page Download   

# cover_url='https://us-east-1.quicksight.aws.amazon.com/sn/dashboards/09acf2c8-158d-47b2-b63a-8cbfd183ad1e'
# page_c= 8
# driver = quicksight_download(driver,page_c,cover_url)
# download_wait(page_d1+page_d2+page_c)


# ## Merge pdf 

# print("Merge pdf..")

# #merge_pdf_name = input("pdf name :")
# merge_pdf_name = "total_pdf_20230407"
# tot_page=page_d1+page_d2+page_c

# merge_pdf(tot_page,merge_pdf_name,result_dir)

# #driver.close()

# end = time()

# print("Done!" + f"{end - start:.5f} sec")

<selenium.webdriver.remote.webelement.WebElement (session="9cf68af8ac95a1dd71cedf81d9b014fc", element="B628268948E151C75A694E9A32DCE4AB_element_272")>

In [72]:
def click_gen_pdf(driver):
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//button[@aria-label="내보내기"]')))
    driver.find_element(By.XPATH, '//button[@aria-label="내보내기"]').click()
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//li[@title="PDF 생성"]')))
    driver.find_element(By.XPATH, '//li[@title="PDF 생성"]').click()

    return driver

def download_toggle_open(driver):
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//button[@aria-label="내보내기"]')))
    driver.find_element(By.XPATH, '//button[@aria-label="내보내기"]').click()
    driver.find_element(By.XPATH, '//li[@data-automation-id="navbar_export_pane_toggle_open"]').click()

    return driver

def clean_page(driver):
     while driver.find_elements(By.ID,'alertCloseButton') != [] :
        driver.find_element(By.ID,'alertCloseButton').click()
     return driver


In [66]:
li=driver.find_elements(By.XPATH,"//div[contains(@class,'sheet-tab-name')]")

len(li)
tttt=[]
for we in li:
    tttt.append(we.text)

In [75]:
page_list=[]
download_toggle_open(driver)
for idx in range(1,19):
        pass_pdf=False
        page_str=driver.find_element(By.XPATH,f'//*[@id="application-content"]/div/div/div[2]/div[1]/div[2]/div/div[1]/div/div/div/div[{idx}]').text
        driver.find_element(By.XPATH,f'//*[@id="application-content"]/div/div/div[2]/div[1]/div[2]/div/div[1]/div/div/div/div[{idx}]').click()
        page_list.append(page_str)
        while pass_pdf == False:
                sleep(2)
                clean_page(driver)
                driver=click_gen_pdf(driver)
                #logger.info(f"Click Generate PDF : {page_str}")
                print(f"Click Generate PDF : {page_str}")
                check_pdf=driver.find_elements(By.XPATH,f"//p[contains(@class,'VegaTypography')] [contains(.,'{page_str}')][contains(.,'파일 이름')]")
                for elem in check_pdf:
                        if elem.text.split(' ')[2] == page_str:
                                pass_pdf=True
                

Click Generate PDF : 4p
Click Generate PDF : 5p
Click Generate PDF : 7p
Click Generate PDF : 8p
Click Generate PDF : 9p
Click Generate PDF : 10p
Click Generate PDF : 11p
Click Generate PDF : 12p
Click Generate PDF : 14p
Click Generate PDF : 15p
Click Generate PDF : 16p
Click Generate PDF : 17p
Click Generate PDF : 18p
Click Generate PDF : 19p
Click Generate PDF : 21p
Click Generate PDF : 22p
Click Generate PDF : 23p
Click Generate PDF : 27p


In [76]:
start=time()
while True:
        while driver.find_elements(By.ID,'alertCloseButton') != [] :
            driver.find_element(By.ID,'alertCloseButton').click()
            
        if len(tttt)==len(driver.find_elements(By.XPATH,f"//div[@data-automation-id='exports_pane_card'][contains(.,'클릭하여 다운로드')]")) :
            print("Click Done!")
            break
        
        elif time()-start > 100 :
            print("Error")
            break

for Webelem in driver.find_elements(By.XPATH,'//*[@data-automation-id="exports_pane_card"]'):
    Webelem.click()
    sleep(0.2)
    

Click Done!


In [197]:
len(tttt)

18

In [9]:
def movePage(driver,url):
    driver.get(url)

def clickGeneratePDF(driver) -> None:
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//button[@aria-label="Export"]')))
    driver.find_element(By.XPATH, '//button[@aria-label="Export"]').click()
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//li[@title="Generate PDF"]')))
    driver.find_element(By.XPATH, '//li[@title="Generate PDF"]').click()

def openDownloadToggle(driver):
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//button[@aria-label="Export"]')))
    driver.find_element(By.XPATH, '//button[@aria-label="Export"]').click()
    driver.find_element(By.XPATH, '//li[@data-automation-id="navbar_export_pane_toggle_open"]').click()

    return True

def clickAlertCloseButton(driver) -> None:
    elems=driver.find_elements(By.ID,'alertCloseButton')
    for elem in elems:
        elem.click()

    return True
        

In [12]:
movePage(url='https://us-east-1.quicksight.aws.amazon.com/sn/dashboards/a7adfe68-ab7e-45f4-a988-94bc41cf9c53',driver=driver)
openDownloadToggle(driver)
clickGeneratePDF(driver)

In [7]:
driver.find_element(By.XPATH,"/html/body/div[4]/div[2]/div/div/div/div[2]/div[1]/div[2]/div/div[1]/div/div/div/div[1]").click()

In [8]:
driver.find_element(By.XPATH,"/html/body/div[4]/div[2]/div/div/div/div[2]/div[1]/div[2]/div/div[1]/div/div/div/div[1]").text

'4p'

In [16]:
driver.find_elements(By.XPATH, "//div[contains(@class,'sheet-tab-name')]")
len(driver.find_elements(By.XPATH, "//div[contains(@class,'sheet-tab-name')]"))

18

In [16]:
driver.get("https://us-east-1.quicksight.aws.amazon.com/sn/dashboards/a7adfe68-ab7e-45f4-a988-94bc41cf9c53")
#WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID,'account-name-input')))
driver.find_element(By.ID,'account-name-input').send_keys('volkswagenkorea').perform()
driver.find_element(By.ID,'account-name-submit-button').click()

sleep(10)

#WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "username-input")))
driver.find_element(By.ID,'username-input').send_keys('syoul@mz.co.kr').perform()
driver.find_element(By.ID,'username-submit-button').click()

sleep(10)
#WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "awsui-input-0")))
driver.find_element(By.ID,'awsui-input-0').send_keys('Soulmz123!').perform()
driver.find_element(By.ID,'password-submit-button').click()

sleep(10)
#WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//button[@aria-label="Export"]')))

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="account-name-input"]"}
  (Session info: chrome=119.0.6045.106); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF64AE382B2+55298]
	(No symbol) [0x00007FF64ADA5E02]
	(No symbol) [0x00007FF64AC605AB]
	(No symbol) [0x00007FF64ACA175C]
	(No symbol) [0x00007FF64ACA18DC]
	(No symbol) [0x00007FF64ACDCBC7]
	(No symbol) [0x00007FF64ACC20EF]
	(No symbol) [0x00007FF64ACDAAA4]
	(No symbol) [0x00007FF64ACC1E83]
	(No symbol) [0x00007FF64AC9670A]
	(No symbol) [0x00007FF64AC97964]
	GetHandleVerifier [0x00007FF64B1B0AAB+3694587]
	GetHandleVerifier [0x00007FF64B20728E+4048862]
	GetHandleVerifier [0x00007FF64B1FF173+4015811]
	GetHandleVerifier [0x00007FF64AED47D6+695590]
	(No symbol) [0x00007FF64ADB0CE8]
	(No symbol) [0x00007FF64ADACF34]
	(No symbol) [0x

In [20]:
from selenium.common.exceptions import *

errors = [NoSuchElementException, ElementNotInteractableException]
wait = WebDriverWait(driver, timeout=10, poll_frequency=.2, ignored_exceptions=errors)

wait.until(lambda d : driver.find_element(By.ID,'account-name-input').send_keys("volkswagenkorea") or True)
wait.until(lambda d : driver.find_element(By.ID,'account-name-submit-button').click() or True)

wait.until(lambda d : driver.find_element(By.ID,'username-input').send_keys('syoul@mz.co.kr') or True)
wait.until(lambda d : driver.find_element(By.ID,'username-submit-button').click() or True)


wait.until(lambda d : driver.find_element(By.ID,'awsui-input-0').send_keys('Soulmz123!') or True)
wait.until(lambda d : driver.find_element(By.ID,'password-submit-button').click() or True)

True

In [12]:
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "username-input")))
driver.find_element(By.ID,'username-input').send_keys('syoul@mz.co.kr')
driver.find_element(By.ID,'username-submit-button').click()

In [9]:
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "awsui-input-0")))
driver.find_element(By.ID,'awsui-input-0').send_keys('Soulmz123!')
driver.find_element(By.ID,'password-submit-button').click()

In [34]:
elems=driver.find_elements(By.ID,'alertCloseButton')
for elem in elems:
    wait.until(lambda d : elem.click() or True)

In [35]:
elems=driver.find_elements(By.XPATH,f"//div[@data-automation-id='exports_pane_card'][contains(.,'Click to download')]")

<selenium.webdriver.remote.webelement.WebElement (session="374d969b2b728f78d9b1dd33d7bfd26d", element="883F86A5DC7502383515989ECF56934D_element_14819")>